![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/onnx/HuggingFace_ONNX_in_Spark_NLP_DistilBertForSequenceClassification.ipynb)

## Import ONNX DistlBertForSequenceClassification models from HuggingFace 🤗  into Spark NLP 🚀

Let's keep in mind a few things before we start 😊

- ONNX support was introduced in  `Spark NLP 5.0.0`, enabling high performance inference for models.
- `DistlBertForSequenceClassification` is only available since in `Spark NLP 5.1.3` and after. So please make sure you have upgraded to the latest Spark NLP release
- You can import DistilBERT models trained/fine-tuned for token classification via `DistilBertForSequenceClassification` or `TFDistilBertForSequenceClassification`. These models are usually under `Token Classification` category and have `bert` in their labels
- Reference: [TFDistilBertForSequenceClassification](https://huggingface.co/transformers/model_doc/distilbert.html#tfdistilbertforsequenceclassification)
- Some [example models](https://huggingface.co/models?filter=distilbert&pipeline_tag=text-classification)

## Export and Save HuggingFace model

- Let's install `transformers` package with the `onnx` extension and it's dependencies. You don't need `onnx` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.29.1`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [ ]:
!pip install -q --upgrade transformers[onnx]==4.29.1 optimum sentencepiece tensorflow==2.11.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 31.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.7/454.7 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 58.9 MB

- HuggingFace has an extension called Optimum which offers specialized model inference, including ONNX. We can use this to import and export ONNX models with `from_pretrained` and `save_pretrained`.
- We'll use [distilbert-base-uncased-finetuned-sst-2-english](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)  model from HuggingFace as an example and load it as a `ORTModelForSequenceClassification`, representing an ONNX model.

In [ ]:
from optimum.onnxruntime import ORTModelForSequenceClassification
import tensorflow as tf

MODEL_NAME = 'distilbert-base-uncased-finetuned-sst-2-english'
ONNX_MODEL = f"onnx_models/{MODEL_NAME}"

ort_model = ORTModelForSequenceClassification.from_pretrained(MODEL_NAME, export=True)

# Save the ONNX model
ort_model.save_pretrained(ONNX_MODEL)

Framework not specified. Using pt to export to ONNX.


Using the export variant default. Available variants are:
	- default: The default ONNX variant.
Using framework PyTorch: 2.0.1+cu118
/usr/local/lib/python3.10/dist-packages/transformers/models/distilbert/modeling_distilbert.py:223: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  mask, torch.tensor(torch.finfo(scores.dtype).min)


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



Let's have a look inside these two directories and see what we are dealing with:

In [ ]:
!ls -l {ONNX_MODEL}

total 262616
-rw-r--r-- 1 root root       735 Oct  3 20:22 config.json
-rw-r--r-- 1 root root 267955711 Oct  3 20:22 model.onnx
-rw-r--r-- 1 root root       125 Oct  3 20:22 special_tokens_map.json
-rw-r--r-- 1 root root       372 Oct  3 20:22 tokenizer_config.json
-rw-r--r-- 1 root root    711396 Oct  3 20:22 tokenizer.json
-rw-r--r-- 1 root root    231508 Oct  3 20:22 vocab.txt


- As you can see, we need to move `vocab.txt` to assets folder which Spark NLP will look for
- We also need `labels` and their `ids` which is saved inside the model's config. We will save this inside `labels.txt`

In [ ]:
!mkdir {ONNX_MODEL}/assets

In [ ]:
# get label2id dictionary
labels = ort_model.config.id2label
# sort the dictionary based on the id
labels = [value for key,value in sorted(labels.items(), reverse=False)]

with open(ONNX_MODEL + '/assets/labels.txt', 'w') as f:
    f.write('\n'.join(labels))

In [ ]:
!mv {ONNX_MODEL}/vocab.txt {ONNX_MODEL}/assets

Voila! We have our `vocab.txt` and `labels.txt` inside assets directory

In [ ]:
!ls -lR {ONNX_MODEL}

onnx_models/distilbert-base-uncased-finetuned-sst-2-english:
total 262392
drwxr-xr-x 2 root root      4096 Oct  3 20:22 assets
-rw-r--r-- 1 root root       735 Oct  3 20:22 config.json
-rw-r--r-- 1 root root 267955711 Oct  3 20:22 model.onnx
-rw-r--r-- 1 root root       125 Oct  3 20:22 special_tokens_map.json
-rw-r--r-- 1 root root       372 Oct  3 20:22 tokenizer_config.json
-rw-r--r-- 1 root root    711396 Oct  3 20:22 tokenizer.json

onnx_models/distilbert-base-uncased-finetuned-sst-2-english/assets:
total 232
-rw-r--r-- 1 root root     17 Oct  3 20:22 labels.txt
-rw-r--r-- 1 root root 231508 Oct  3 20:22 vocab.txt


## Import and Save DistilBertForSequenceClassification in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [ ]:
! wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2023-10-03 20:22:10--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2023-10-03 20:22:10--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1191 (1.2K) [text/plain]
Saving to: ‘STDOUT’

-                     0%[                    ]       0  --.-KB/s               Installing PySpark 3.2.3 and Spark NLP 5.1.2
setup Colab for PySpark 3.2.3 an

Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

print("Apache Spark version: {}".format(spark.version))

Apache Spark version: 3.2.3


- Let's use `loadSavedModel` functon in `DistilBertForSequenceClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `DistilBertForSequenceClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [ ]:
from sparknlp.annotator import *
from sparknlp.base import *

sequenceClassifier = DistilBertForSequenceClassification.loadSavedModel(
     ONNX_MODEL,
     spark
 )\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")\
  .setCaseSensitive(True)\
  .setMaxSentenceLength(128)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [ ]:
sequenceClassifier.write().overwrite().save("./{}_spark_nlp_onnx".format(ONNX_MODEL))

Let's clean up stuff we don't need anymore

In [ ]:
!rm -rf {ONNX_MODEL}

Awesome 😎  !

This is your AlbertForSequenceClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [ ]:
! ls -l {ONNX_MODEL}_spark_nlp_onnx

total 261724
-rw-r--r-- 1 root root 267996775 Oct  3 20:28 distilbert_classification_onnx
drwxr-xr-x 4 root root      4096 Oct  3 20:28 fields
drwxr-xr-x 2 root root      4096 Oct  3 20:28 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny AlbertForSequenceClassification model 😊

In [ ]:
sequenceClassifier_loaded = DistilBertForSequenceClassification.load("./{}_spark_nlp_onnx".format(ONNX_MODEL))\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")

You can see what labels were used to train this model via `getClasses` function:

In [ ]:
# .getClasses was introduced in spark-nlp==3.4.0
sequenceClassifier_loaded.getClasses()

['NEGATIVE', 'POSITIVE']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [ ]:
from sparknlp.annotator import *
from sparknlp.base import *

document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    sequenceClassifier_loaded
])

# couple of simple examples
example = spark.createDataFrame([["I love you!"], ['I feel lucky to be here.'], ['I hate her!']]).toDF("text")

result = pipeline.fit(example).transform(example)

# result is a DataFrame
result.select("text", "class.result").show()

+--------------------+----------+
|                text|    result|
+--------------------+----------+
|         I love you!|[POSITIVE]|
|I feel lucky to b...|[POSITIVE]|
|         I hate her!|[NEGATIVE]|
+--------------------+----------+



That's it! You can now go wild and use hundreds of `DistlBertForSequenceClassification` models from HuggingFace 🤗 in Spark NLP 🚀
